# 댓글만 가져오기


In [ ]:
#talkTab > a

from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import pandas as pd

# 함수 정의
# 화면 스크롤 하는 함수
def scroll_to(height):
    current_scroll_position = driver.execute_script("return window.pageYOffset;")       # 현재 위치 가져오기
    new_scroll_position = current_scroll_position + height          # 지금 위치에서 height 추가한 새로운 스크롤 위치
    driver.execute_script(f'window.scrollTo(0,{new_scroll_position});')     # 새로운 스크롤 위치로 이동
    time.sleep(1)

# 지역 버튼 클릭하는 함수
def click_region(location_number):
    # 강원 (32) / 충북 (33) / 울산 (7) / 대전(3) / 각자 지역 참고하세요
    button_path = driver.find_element(By.XPATH, f'//*[@id="{location_number}"]/button')
    button_path.click()
    time.sleep(2)

tour = []

service=webdriver.chrome.service.Service('chromedriver.exe')
driver=webdriver.Chrome(service=service)


driver.get('https://korean.visitkorea.or.kr/list/travelinfo.do?service=ms#ms^0^32^All^e6875575-2cc2-43ba-9651-28d31a7b3e23,651c5b95-a5b3-11e8-8165-020027310001,c24d515f-3202-45e5-834e-1a091901aeff,d3fd4d9f-fbd4-430f-b5d5-291b4d9920be,3f36ca4b-6f45-45cb-9042-265c96a4868c,23bc02b8-da01-41bf-8118-af882436cd3c,2d4f4e06-2d37-4e54-ad5c-172add6e6680,9668f0f1-8afe-4526-8007-503bd02fd6d8,0f29b431-75ac-4ab4-a892-b247d516b31d,640d3489-8fc3-11e8-8165-020027310001,1601b0a3-144e-40b7-95b4-b946e537a25b,1c981ad4-7834-11e8-82c8-020027310001,266bf7a0-cbab-4bb4-b800-d7edd5642180,cdd12d65-1f38-4829-a1be-bf235a0fb3f2^1^^1^#%EA%B0%95%EC%9B%90')
time.sleep(5)

# 지역 번호 바꾸세요!!!!
region_num = 3    # 대전
click_region(region_num)

# 전체 결과 수 확인하고 총 페이지 수 확인
total_results = int(driver.find_element(By.XPATH,'//*[@id="totalCnt"]').text.replace(",",""))
total_pages = total_results//10 +1     # 한 페이지당 10개씩 나와요 그래서 총 페이지 수 total_results // 10 + 1 해줘야함


# 페이지마다 결과물 10개씩 나옴
# 그거 다 받아서 for 문으로 돌리면서 하나씩 들어가서 정보 crawling
for i in range(total_pages):
    # 해당 페이지에 보여지는 결과물 10개 contents에 저장
    locations= driver.find_elements(By.CSS_SELECTOR,'div.box_leftType1> ul > li')

    # 결과물 1개씩 클릭해서 정보 크롤링
    for idx,location in enumerate(locations):
        # 정보 저장할 딕셔너리 초기화
        location_data = {}
        # 장소 이름 클릭할 수 있도록 button 찾아주고 클릭
        # locations = driver.find_elements(By.CSS_SELECTOR, '#contents > div:nth-child(2) > div:nth-child(1) > ul > li')
        # link = locations[idx].find_element(By.TAG_NAME, 'a')
        link = location.find_element(By.TAG_NAME,'a')
        link.click()
        time.sleep(2)

        # 페이지로 이동한 후 데이터 얻기
        # 장소 이름 저장
        title = driver.find_element(By.ID, 'topTitle')
        location_data['place'] = title.text

        # 상세 정보 보이는 쪽으로 스크롤
        scroll_to(900)
        time.sleep(1)

        # 상세정보 찾아서 text 저장
        info = driver.find_element(By.CSS_SELECTOR, 'div.inr_wrap > div.inr > p')
        location_data['info'] = info.text
        time.sleep(1)

        # 상세 항목 저장
        # 아래쪽으로 이동
        scroll_to(600)
        # 더보기 있으면 클릭
        try:
            driver.find_element(By.XPATH, '//*[@id="detailinfoview"]/div/div[2]/button').click()
        except:
            pass
        
        # 장소 상세 정보 다운로드
        additional_contents = driver.find_elements(By.CSS_SELECTOR, 'div.wrap_contView div.inr li')
        for additional_content in additional_contents:
            key = additional_content.find_element(By.CSS_SELECTOR, 'strong').text
            value = additional_content.find_element(By.CSS_SELECTOR, 'span').text
            location_data[key] = value
        

        # 리뷰 데이터 크롤링
        button = driver.find_element(By.CSS_SELECTOR,'#talkTab > a')
        button.click()
        time.sleep(1)

        review_list = []    # 리뷰 담을 변수 초기화
        scroll_to(600)
        time.sleep(2)
        try:
            review_count = int(driver.find_element(By.ID, 'commentCount').text)
            print(review_count)

            # 리뷰 개수 세기
            count = min( 10, review_count)
            # 리뷰 개수 0개이면 뒤로가기 
            if count == 0 :
                driver.back()
                continue
            # 리뷰 2개 이하면 리뷰 추출 (더보기 안 눌러도 다 보여서 굳이 더보기 누를 필요 없음)
            elif count <=2:
                review_contents = driver.find_elements(By.CSS_SELECTOR, '#commentArea > li')
                print(review_contents)
            # 리뷰 3개이상 7개 이하면 더보기 한 번 눌러야함
            elif count <=7:
                scroll_to(300)
                driver.find_element(By.XPATH, '//*[@id="commentMore"]/button').click() # 더보기 버튼 누르기     # 리뷰 더보기 클릭
                time.sleep(2)
                review_contents = driver.find_elements(By.CSS_SELECTOR, '#commentArea > li')
                print(review_contents)
            # 리뷰 8개 이상이면 더보기 2번 눌러야함
            else:
                scroll_to(300)
                driver.find_element(By.XPATH, '//*[@id="commentMore"]/button').click()      # 리뷰 더보기 클릭
                time.sleep(1.5)
                scroll_to(700)
                try:
                    driver.find_element(By.XPATH, '//*[@id="commentMore"]/button').click()      # 리뷰 더보기 클릭
                    time.sleep(2)
                except:
                    pass
                review_contents = driver.find_elements(By.CSS_SELECTOR, '#commentArea > li')
                print(review_contents)

            button.click()
            scroll_to(600)
            # 리뷰 컨텐츠들을 하나씩 꺼내어서 review_text 추출
            for review_content in review_contents:
                review_list.append(review_content.find_element(By.CLASS_NAME, 'review_text').text)
                scroll_to(120)
            reviews = " ".join(review_list)

            # 리뷰 정보 딕셔너리에 저장
            location_data['review'] = reviews

        except: # 오류 발생시 오류라고 저장하고 넘어가게 해줌
            location_data['review'] = '오류'

        finally:
            # 투어 리스트에 location_data 전체 저장
            tour.append(location_data)

            # 정보 다 크롤링한 후 뒤로 가기
            driver.back()
            scroll_to(120)
            time.sleep(2)

    # 아래로 내려가기
    scroll_to(200)
    time.sleep(1)

    # 다음 페이지 버튼 누르기
    page_button_path = driver.find_elements(By.CSS_SELECTOR, '.page_box > a')
    page_button_path[-2].click()
    time.sleep(3)
    if i==0:
        break

pd.DataFrame(tour).to_csv(f'{region_num}번 지역 데이터')
time.sleep(1)
driver.close()

1
[<selenium.webdriver.remote.webelement.WebElement (session="a629a94a02818eee7913799e8523594a", element="f.0CCE3FB07EC6BB7977212BE50D5BD076.d.FE3AB6CC495B60A577698782B873ADCD.e.13359")>]
2
[<selenium.webdriver.remote.webelement.WebElement (session="a629a94a02818eee7913799e8523594a", element="f.0CCE3FB07EC6BB7977212BE50D5BD076.d.C41C113EEE0A72A2FBC1286F718D0B0E.e.25422")>, <selenium.webdriver.remote.webelement.WebElement (session="a629a94a02818eee7913799e8523594a", element="f.0CCE3FB07EC6BB7977212BE50D5BD076.d.C41C113EEE0A72A2FBC1286F718D0B0E.e.25445")>]
4
[<selenium.webdriver.remote.webelement.WebElement (session="a629a94a02818eee7913799e8523594a", element="f.0CCE3FB07EC6BB7977212BE50D5BD076.d.454FCC09B618476CDEF0DE06146B3E51.e.37623")>, <selenium.webdriver.remote.webelement.WebElement (session="a629a94a02818eee7913799e8523594a", element="f.0CCE3FB07EC6BB7977212BE50D5BD076.d.454FCC09B618476CDEF0DE06146B3E51.e.37643")>, <selenium.webdriver.remote.webelement.WebElement (session="a629a94

In [ ]:
#talkTab > a

from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import pandas as pd

# 함수 정의
# 화면 스크롤 하는 함수
def scroll_to(height):
    current_scroll_position = driver.execute_script("return window.pageYOffset;")       # 현재 위치 가져오기
    new_scroll_position = current_scroll_position + height          # 지금 위치에서 height 추가한 새로운 스크롤 위치
    driver.execute_script(f'window.scrollTo(0,{new_scroll_position});')     # 새로운 스크롤 위치로 이동
    time.sleep(1)

# 지역 버튼 클릭하는 함수
def click_region(location_number):
    # 강원 (32) / 충북 (33) / 울산 (7) / 대전(3) / 각자 지역 참고하세요
    button_path = driver.find_element(By.XPATH, f'//*[@id="{location_number}"]/button')
    button_path.click()
    time.sleep(2)

tour = []

service=webdriver.chrome.service.Service('chromedriver.exe')
driver=webdriver.Chrome(service=service)


driver.get('https://korean.visitkorea.or.kr/list/travelinfo.do?service=ms#ms^0^32^All^e6875575-2cc2-43ba-9651-28d31a7b3e23,651c5b95-a5b3-11e8-8165-020027310001,c24d515f-3202-45e5-834e-1a091901aeff,d3fd4d9f-fbd4-430f-b5d5-291b4d9920be,3f36ca4b-6f45-45cb-9042-265c96a4868c,23bc02b8-da01-41bf-8118-af882436cd3c,2d4f4e06-2d37-4e54-ad5c-172add6e6680,9668f0f1-8afe-4526-8007-503bd02fd6d8,0f29b431-75ac-4ab4-a892-b247d516b31d,640d3489-8fc3-11e8-8165-020027310001,1601b0a3-144e-40b7-95b4-b946e537a25b,1c981ad4-7834-11e8-82c8-020027310001,266bf7a0-cbab-4bb4-b800-d7edd5642180,cdd12d65-1f38-4829-a1be-bf235a0fb3f2^1^^1^#%EA%B0%95%EC%9B%90')
time.sleep(5)

# 지역 번호 바꾸세요!!!!
region_num = 7    # 대전
click_region(region_num)

# 전체 결과 수 확인하고 총 페이지 수 확인
total_results = int(driver.find_element(By.XPATH,'//*[@id="totalCnt"]').text.replace(",",""))
total_pages = total_results//10 +1     # 한 페이지당 10개씩 나와요 그래서 총 페이지 수 total_results // 10 + 1 해줘야함


# 페이지마다 결과물 10개씩 나옴
# 그거 다 받아서 for 문으로 돌리면서 하나씩 들어가서 정보 crawling
for _ in range(total_pages):
    # 해당 페이지에 보여지는 결과물 10개 contents에 저장
    
    location_counts = len(driver.find_elements(By.CSS_SELECTOR, '#contents > div:nth-child(2) > div:nth-child(1) > ul > li'))
    # 결과물 1개씩 클릭해서 정보 크롤링
    for idx in range(location_counts):
        # 정보 저장할 딕셔너리 초기화
        location_data = {}
        # 장소 이름 클릭할 수 있도록 button 찾아주고 클릭
        locations = driver.find_elements(By.CSS_SELECTOR, '#contents > div:nth-child(2) > div:nth-child(1) > ul > li')
        link = locations[idx].find_element(By.TAG_NAME, 'a')
        # link = location.find_element(By.TAG_NAME,'a')
        # link = location.find_element(By.CSS_SELECTOR, '.tit > a')
        link.click()
        time.sleep(2)

        # 페이지로 이동한 후 데이터 얻기
        # 장소 이름 저장
        title = driver.find_element(By.ID, 'topTitle')
        location_data['place'] = title.text

        # 상세 정보 보이는 쪽으로 스크롤
        scroll_to(900)
        time.sleep(1)

        # 상세정보 찾아서 text 저장
        info = driver.find_element(By.CSS_SELECTOR, 'div.inr_wrap > div.inr > p')
        location_data['info'] = info.text
        time.sleep(1)

        # 상세 항목 저장
        # 아래쪽으로 이동
        scroll_to(600)
        # 더보기 있으면 클릭
        try:
            driver.find_element(By.XPATH, '//*[@id="detailinfoview"]/div/div[2]/button').click()
        except:
            pass
        
        # 장소 상세 정보 다운로드
        additional_contents = driver.find_elements(By.CSS_SELECTOR, 'div.wrap_contView div.inr li')
        for additional_content in additional_contents:
            key = additional_content.find_element(By.CSS_SELECTOR, 'strong').text
            value = additional_content.find_element(By.CSS_SELECTOR, 'span').text
            location_data[key] = value
        

        # 리뷰 데이터 크롤링
        button = driver.find_element(By.CSS_SELECTOR,'#talkTab > a')
        button.click()
        time.sleep(1)

        review_list = []    # 리뷰 담을 변수 초기화
        scroll_to(600)
        time.sleep(2)
        try:
            review_count = int(driver.find_element(By.ID, 'commentCount').text)
            print(review_count)

            # 리뷰 개수 세기
            count = min( 10, review_count)
            # 리뷰 개수 0개이면 뒤로가기 
            if count == 0 :
                driver.back()
                continue
            # 리뷰 2개 이하면 리뷰 추출 (더보기 안 눌러도 다 보여서 굳이 더보기 누를 필요 없음)
            elif count <=2:
                review_contents = driver.find_elements(By.CSS_SELECTOR, '#commentArea > li')
                print(review_contents)
            # 리뷰 3개이상 7개 이하면 더보기 한 번 눌러야함
            elif count <=7:
                scroll_to(300)
                driver.find_element(By.XPATH, '//*[@id="commentMore"]/button').click() # 더보기 버튼 누르기     # 리뷰 더보기 클릭
                time.sleep(2)
                review_contents = driver.find_elements(By.CSS_SELECTOR, '#commentArea > li')
                print(review_contents)
            # 리뷰 8개 이상이면 더보기 2번 눌러야함
            else:
                scroll_to(300)
                driver.find_element(By.XPATH, '//*[@id="commentMore"]/button').click()      # 리뷰 더보기 클릭
                time.sleep(1.5)
                scroll_to(700)
                try:
                    driver.find_element(By.XPATH, '//*[@id="commentMore"]/button').click()      # 리뷰 더보기 클릭
                    time.sleep(2)
                except:
                    pass
                review_contents = driver.find_elements(By.CSS_SELECTOR, '#commentArea > li')
                print(review_contents)

            button.click()
            scroll_to(600)
            # 리뷰 컨텐츠들을 하나씩 꺼내어서 review_text 추출
            for review_content in review_contents:
                review_list.append(review_content.find_elements(By.CLASS_NAME, 'review_text')[-1].text)
                scroll_to(120)
            reviews = " ".join(review_list)

            # 리뷰 정보 딕셔너리에 저장
            location_data['review'] = reviews

        except: # 오류 발생시 오류라고 저장하고 넘어가게 해줌
            location_data['review'] = '오류'

        finally:
            # 투어 리스트에 location_data 전체 저장
            tour.append(location_data)

            # 정보 다 크롤링한 후 뒤로 가기
            driver.back()
            scroll_to(120)
            time.sleep(2)

    # 아래로 내려가기
    scroll_to(200)
    time.sleep(1)

    # 다음 페이지 버튼 누르기
    page_button_path = driver.find_elements(By.CSS_SELECTOR, '.page_box > a')
    page_button_path[-2].click()
    time.sleep(3)

pd.DataFrame(tour).to_csv(f'{region_num}번 지역 데이터')
time.sleep(1)
driver.close()

6
[<selenium.webdriver.remote.webelement.WebElement (session="f81ee618b41e9ad4c971545ffd809c4f", element="f.6E22A12ED11425F2D713447EB70DB91C.d.51491433768AB9C52711DEAE208DCD98.e.13417")>, <selenium.webdriver.remote.webelement.WebElement (session="f81ee618b41e9ad4c971545ffd809c4f", element="f.6E22A12ED11425F2D713447EB70DB91C.d.51491433768AB9C52711DEAE208DCD98.e.13440")>, <selenium.webdriver.remote.webelement.WebElement (session="f81ee618b41e9ad4c971545ffd809c4f", element="f.6E22A12ED11425F2D713447EB70DB91C.d.51491433768AB9C52711DEAE208DCD98.e.13752")>, <selenium.webdriver.remote.webelement.WebElement (session="f81ee618b41e9ad4c971545ffd809c4f", element="f.6E22A12ED11425F2D713447EB70DB91C.d.51491433768AB9C52711DEAE208DCD98.e.13772")>, <selenium.webdriver.remote.webelement.WebElement (session="f81ee618b41e9ad4c971545ffd809c4f", element="f.6E22A12ED11425F2D713447EB70DB91C.d.51491433768AB9C52711DEAE208DCD98.e.13798")>, <selenium.webdriver.remote.webelement.WebElement (session="f81ee618b41e9

In [8]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import pandas as pd

location_data = {}
# 함수 정의
# 화면 스크롤 하는 함수
def scroll_to(height):
    current_scroll_position = driver.execute_script("return window.pageYOffset;")       # 현재 위치 가져오기
    new_scroll_position = current_scroll_position + height          # 지금 위치에서 height 추가한 새로운 스크롤 위치
    driver.execute_script(f'window.scrollTo(0,{new_scroll_position});')     # 새로운 스크롤 위치로 이동
    time.sleep(1)

# 지역 버튼 클릭하는 함수
def click_region(location_number):
    # 강원 (32) / 충북 (33) / 울산 (7) / 대전(3) / 각자 지역 참고하세요
    button_path = driver.find_element(By.XPATH, f'//*[@id="{location_number}"]/button')
    button_path.click()
    time.sleep(2)

tour = []

service=webdriver.chrome.service.Service('chromedriver.exe')
driver=webdriver.Chrome(service=service)


driver.get('https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=76ea6c77-5d20-475d-839e-2dd9da85d201')
time.sleep(5)

# 페이지로 이동한 후 데이터 얻기
# 장소 이름 저장
title = driver.find_element(By.ID, 'topTitle')
location_data['place'] = title.text

# 상세 정보 보이는 쪽으로 스크롤
scroll_to(900)
time.sleep(1)

# 상세정보 찾아서 text 저장
info = driver.find_element(By.CSS_SELECTOR, 'div.inr_wrap > div.inr > p')
location_data['info'] = info.text
time.sleep(1)

# 상세 항목 저장
# 아래쪽으로 이동
scroll_to(600)
# 더보기 있으면 클릭
try:
    driver.find_element(By.XPATH, '//*[@id="detailinfoview"]/div/div[2]/button').click()
except:
    pass

# 장소 상세 정보 다운로드
additional_contents = driver.find_elements(By.CSS_SELECTOR, 'div.wrap_contView div.inr li')
for additional_content in additional_contents:
    key = additional_content.find_element(By.CSS_SELECTOR, 'strong').text
    value = additional_content.find_element(By.CSS_SELECTOR, 'span').text
    location_data[key] = value


# 리뷰 데이터 크롤링
review_list = []    # 리뷰 담을 변수 초기화
scroll_to(600)
try:
    review_count = int(driver.find_element(By.ID, 'commentCount').text)

    # 리뷰 개수 세기
    count = min( 10, review_count)
    # 리뷰 개수 0개이면 뒤로가기 
    if count == 0 :
        pass
    # 리뷰 2개 이하면 리뷰 추출 (더보기 안 눌러도 다 보여서 굳이 더보기 누를 필요 없음)
    elif count <=2:
        review_contents = driver.find_elements(By.CSS_SELECTOR, '#commentArea > li')
    # 리뷰 3개이상 7개 이하면 더보기 한 번 눌러야함
    elif count <=7:
        scroll_to(300)
        driver.find_element(By.XPATH, '//*[@id="commentMore"]/button').click() # 더보기 버튼 누르기     # 리뷰 더보기 클릭
        time.sleep(2)
        review_contents = driver.find_elements(By.CSS_SELECTOR, '#commentArea > li')
    # 리뷰 8개 이상이면 더보기 2번 눌러야함
    else:
        scroll_to(300)
        driver.find_element(By.XPATH, '//*[@id="commentMore"]/button').click()      # 리뷰 더보기 클릭
        time.sleep(1)
        scroll_to(700)
        try:
            driver.find_element(By.XPATH, '//*[@id="commentMore"]/button').click()      # 리뷰 더보기 클릭
        except:
            pass
        review_contents = driver.find_elements(By.CSS_SELECTOR, '.commentArea li')
    
    scroll_to(200)

    # 리뷰 컨텐츠들을 하나씩 꺼내어서 review_text 추출
    for review_content in review_contents:
        scroll_to(187)
        try:
            review_content.find_element(By.CSS_SELECTOR, 'p > button').click()
            review_list.append(review_content.find_elements(By.CLASS_NAME, 'review_text')[-1].text)
            print(review_content.find_elements(By.CLASS_NAME, 'review_text')[-1].text)
            print(review_content.find_elements(By.CLASS_NAME, 'review_text'))
        except:
            review_list.append(review_content.find_element(By.CLASS_NAME,'review_text').text)
    reviews = " ".join(review_list)
    print(reviews)

    # 리뷰 정보 딕셔너리에 저장
    location_data['review'] = reviews

except: # 오류 발생시 오류라고 저장하고 넘어가게 해줌
    location_data['review']= '오류'

finally:
    # 투어 리스트에 location_data 전체 저장
    tour.append(location_data)
    print(location_data)


time.sleep(1)
driver.close()


입구쪽에 올리브영도 있고 스타벅스도 있고 헌혈카페도 있고 최근에는 KFC도 오픈했어요. 그 외에도 많은 수의 카페, 식당, 술집, 편의점 등등이 깔끔하게 되어있는 먹자골목 같은 곳입니다. 식사, 후식, 약간의 쇼핑, 노는것까지 한군데에서 해결 가능 합니다. 중심 광장쪽에는 주로 아이들이 전동차를 타고 놀고 가끔 버스킹같이 노래도 부르고 행사도 합니다. 근데 여기가 차가 못다니는 곳은 아니지만...
더보기 마치광장은 먹거리와 놀이를 같이할수있는 우리동네 놀이터 저녁에 어린이들도많이나와 노는장송ㅂ니다 우리 동네, 제 삶의 터전을 관광지 소개란에서 만나니까 감회가 새롭고 반갑네요. 대전 분들 사이에서는 이미 '먹자골목'으로 유명했던 대전 서구 관저동의 또 다른 랜드마크 같은 곳이랍니다. 생긴 지는 10년쯤 된 것 같아요. 말 그대로 광장에 충실한 장소예요. 벼룩시장(플리 마켓)이나 버스킹 공연이 종종 열리고요. 주말에는 아이들이 전동차를 빌려서 원을 뺑 도는 모습을 보노라면 장관이에요. 마...
더보기
{'place': '마치광장', 'info': '마치광장은 대전시 서구 관저 5 지구에 있다. 광장 근처는 많은 사람이 밀집하는 관저동의 대표 중심 상권이다. 음식점이 많아 주중이나 주말 가릴 것 없이 많은 사람이 찾는다. 광장 주변에는 아이들을 위한 전동자동차 대여 상점이 많으며, 전동자동차를 즐기는 아이들이 많다.', '문의 및 안내': '', '주소': '대전광역시 서구 구봉로131번길 27', '이용시간': '상시 개방', '휴일': '연중무휴', '주차': '불가능', '입장료': '무료', 'review': "입구쪽에 올리브영도 있고 스타벅스도 있고 헌혈카페도 있고 최근에는 KFC도 오픈했어요. 그 외에도 많은 수의 카페, 식당, 술집, 편의점 등등이 깔끔하게 되어있는 먹자골목 같은 곳입니다. 식사, 후식, 약간의 쇼핑, 노는것까지 한군데에서 해결 가능 합니다. 중심 광장쪽에는 주로 아이들이 전동차를 타고 놀고 가끔 버스킹같이 노래도 부르고 행사도 합니다.

In [ ]:
review_contents

[<selenium.webdriver.remote.webelement.WebElement (session="28b3d6cdc9fa03ccbe213f8d78d2ad50", element="f.1FFC9D2873B9EF782CBA7F8A7818595F.d.103EEC4F973763A31BB17679E568E3E9.e.5555")>,
 <selenium.webdriver.remote.webelement.WebElement (session="28b3d6cdc9fa03ccbe213f8d78d2ad50", element="f.1FFC9D2873B9EF782CBA7F8A7818595F.d.103EEC4F973763A31BB17679E568E3E9.e.5584")>,
 <selenium.webdriver.remote.webelement.WebElement (session="28b3d6cdc9fa03ccbe213f8d78d2ad50", element="f.1FFC9D2873B9EF782CBA7F8A7818595F.d.103EEC4F973763A31BB17679E568E3E9.e.5834")>]

In [7]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import pandas as pd

# 함수 정의
# 화면 스크롤 하는 함수
def scroll_to(height):
    current_scroll_position = driver.execute_script("return window.pageYOffset;")       # 현재 위치 가져오기
    new_scroll_position = current_scroll_position + height          # 지금 위치에서 height 추가한 새로운 스크롤 위치
    driver.execute_script(f'window.scrollTo(0,{new_scroll_position});')     # 새로운 스크롤 위치로 이동
    time.sleep(1)

# 지역 버튼 클릭하는 함수
def click_region(location_number):
    # 전남 id = 38 | 서울 id = 1 | 대구 id = 4
    button_path = driver.find_element(By.XPATH, f'//*[@id="{location_number}"]/button')
    button_path.click()
    time.sleep(2)

tour = []

service=webdriver.chrome.service.Service('chromedriver.exe')
driver=webdriver.Chrome(service=service)


driver.get('https://korean.visitkorea.or.kr/list/travelinfo.do?service=ms#ms^0^32^All^e6875575-2cc2-43ba-9651-28d31a7b3e23,651c5b95-a5b3-11e8-8165-020027310001,c24d515f-3202-45e5-834e-1a091901aeff,d3fd4d9f-fbd4-430f-b5d5-291b4d9920be,3f36ca4b-6f45-45cb-9042-265c96a4868c,23bc02b8-da01-41bf-8118-af882436cd3c,2d4f4e06-2d37-4e54-ad5c-172add6e6680,9668f0f1-8afe-4526-8007-503bd02fd6d8,0f29b431-75ac-4ab4-a892-b247d516b31d,640d3489-8fc3-11e8-8165-020027310001,1601b0a3-144e-40b7-95b4-b946e537a25b,1c981ad4-7834-11e8-82c8-020027310001,266bf7a0-cbab-4bb4-b800-d7edd5642180,cdd12d65-1f38-4829-a1be-bf235a0fb3f2^1^^1^#%EA%B0%95%EC%9B%90')
time.sleep(5)

# 지역 번호 바꾸세요!!!!
region_num = 1
click_region(region_num)

# 전체 결과 수 확인하고 총 페이지 수 확인
total_results = int(driver.find_element(By.XPATH,'//*[@id="totalCnt"]').text.replace(",",""))
total_pages = total_results//10 +1     # 한 페이지당 10개씩 나와요 그래서 총 페이지 수 total_results // 10 + 1 해줘야함


# 페이지마다 결과물 10개씩 나옴
# 그거 다 받아서 for 문으로 돌리면서 하나씩 들어가서 정보 crawling
for _ in range(total_pages):
    # 해당 페이지에 보여지는 결과물 10개 contents에 저장
    
    location_counts = len(driver.find_elements(By.CSS_SELECTOR, '#contents > div:nth-child(2) > div:nth-child(1) > ul > li'))
    # 결과물 1개씩 클릭해서 정보 크롤링
    for idx in range(location_counts):
        # 정보 저장할 딕셔너리 초기화
        location_data = {}
        # 장소 이름 클릭할 수 있도록 button 찾아주고 클릭
        time.sleep(2)
        locations = driver.find_elements(By.CSS_SELECTOR, '#contents > div:nth-child(2) > div:nth-child(1) > ul > li')
        link = locations[idx].find_element(By.TAG_NAME, 'a')
        link.click()
        time.sleep(2)

        # 페이지로 이동한 후 데이터 얻기
        # 장소 이름 저장
        title = driver.find_element(By.ID, 'topTitle')
        location_data['place'] = title.text

        # 상세 정보 보이는 쪽으로 스크롤
        scroll_to(900)
        time.sleep(1)

        # 상세정보 찾아서 text 저장
        info = driver.find_element(By.CSS_SELECTOR, 'div.inr_wrap > div.inr > p')
        location_data['info'] = info.text
        time.sleep(1)

        # 상세 항목 저장
        # 아래쪽으로 이동
        scroll_to(600)
        # 더보기 있으면 클릭
        try:
            driver.find_element(By.XPATH, '//*[@id="detailinfoview"]/div/div[2]/button').click()
        except:
            pass
        
        # 장소 상세 정보 다운로드
        additional_contents = driver.find_elements(By.CSS_SELECTOR, 'div.wrap_contView div.inr li')
        for additional_content in additional_contents:
            key = additional_content.find_element(By.CSS_SELECTOR, 'strong').text
            value = additional_content.find_element(By.CSS_SELECTOR, 'span').text
            location_data[key] = value
        
        # 리뷰 데이터 크롤링
        review_list = []    # 리뷰 담을 변수 초기화
        scroll_to(600)
        try:
            review_count = int(driver.find_element(By.ID, 'commentCount').text)

            # 리뷰 개수 세기
            count = min( 10, review_count)
            # 리뷰 개수 0개이면 뒤로가기 
            if count == 0 :
                driver.back()
                continue
            # 리뷰 2개 이하면 리뷰 추출 (더보기 안 눌러도 다 보여서 굳이 더보기 누를 필요 없음)
            elif count <=2:
                review_contents = driver.find_elements(By.CSS_SELECTOR, '#commentArea > li')
            # 리뷰 3개이상 7개 이하면 더보기 한 번 눌러야함
            elif count <=7:
                scroll_to(300)
                driver.find_element(By.XPATH, '//*[@id="commentMore"]/button').click() # 더보기 버튼 누르기     # 리뷰 더보기 클릭
                time.sleep(2)
                review_contents = driver.find_elements(By.CSS_SELECTOR, '#commentArea > li')
            # 리뷰 8개 이상이면 더보기 2번 눌러야함
            else:
                scroll_to(300)
                driver.find_element(By.XPATH, '//*[@id="commentMore"]/button').click()      # 리뷰 더보기 클릭
                time.sleep(1)
                scroll_to(700)
                try:
                    driver.find_element(By.XPATH, '//*[@id="commentMore"]/button').click()      # 리뷰 더보기 클릭
                except:
                    pass
                review_contents = driver.find_elements(By.CSS_SELECTOR, '.commentArea li')
            
            scroll_to(200)

            # # 리뷰 컨텐츠들을 하나씩 꺼내어서 review_text 추출
            # for review_content in review_contents:
            #     scroll_to(187)
            #     review_list.append(review_content.find_element(By.CLASS_NAME, 'review_text').text)
            # reviews = " ".join(review_list)

            for review_content in review_contents:
                # 리뷰 안에 '더보기' 버튼이 있으면 클릭
                more_button = review_content.find_elements(By.CSS_SELECTOR, ".more")
                if more_button:
                    driver.execute_script("arguments[0].scrollIntoView(true);", more_button[0])
                    driver.execute_script("arguments[0].click();", more_button[0])
                    time.sleep(0.5)

                # 리뷰 본문 추출
                try:
                    review_text = review_content.find_element(By.CLASS_NAME, "review_text").text.strip()
                except:
                    review_text = ""
                review_list.append(review_text)

            # 리뷰 전체 합치기 
            reviews = " ".join(review_list).strip()

            # 딕셔너리에 저장
            location_data['review'] = reviews if reviews else "리뷰 없음"



            # # 리뷰 컨텐츠들을 하나씩 꺼내어서 review_text 추출
            # for review_content in review_contents:
            # # 리뷰 안에 '더보기' 버튼이 있으면 클릭
            #     more_button = review_content.find_elements(By.CSS_SELECTOR, ".more")
            #     if more_button:
            #         driver.execute_script("arguments[0].scrollIntoView(true);", more_button[0]) 
            #         driver.execute_script("arguments[0].click();", more_button[0])
            #         time.sleep(0.5)

            #     # 리뷰 본문 추출
            #     review_text = review_content.find_element(By.CLASS_NAME, "review_text").text
            #     review_list.append(review_text)

            # # 리뷰 전체 합치기
            # reviews = " ".join(review_list)

            # # 리뷰 정보 딕셔너리에 저장
            # location_data['review'] = reviews

        except: # 오류 발생시 오류라고 저장하고 넘어가게 해줌
            location_data['review']= '오류'

        finally:
            # 투어 리스트에 location_data 전체 저장
            tour.append(location_data)
            print(location_data)

            # 정보 다 크롤링한 후 뒤로 가기
            driver.back()
            scroll_to(120)
            time.sleep(2)

    # 아래로 내려가기
    scroll_to(200)
    time.sleep(1)

    # 다음 페이지 버튼 누르기
    page_button_path = driver.find_elements(By.CSS_SELECTOR, '.page_box > a')
    page_button_path[-2].click()
    time.sleep(3)

pd.DataFrame(tour).to_csv(f'{region_num}번 지역 데이터')
time.sleep(1)
driver.close()

{'place': '식민지역사박물관', 'info': '식민지역사박물관은 일본제국주의 침탈의 역사와 그에 부역한 친일파의 죄상, 빛나는 항일투쟁의 역사를 기록하고 전시하는 최초의 일제강점기 전문 역사박물관이다. 이 소중한 역사문화공간은 해외동포를 포함한 수많은 시민들의 성금과 기증자료에 힘입어 마련되었다. 이곳은 총 5층 건물로 1층은 전시, 뮤지엄숍, 이벤트홀 등 복합문화공간이고 2층은 기록영상을 관람할 수 있고 식민지 생활상 체험을 할 수 있는 곳이다. 3층은 민족문제연구소와 4층은 자료실, 5층은 강의실로 사용한다. 또한 옥상에는 남산과 용산 일대를 조망할 수 있는 전망대와 잠시 휴식을 취할 수 있는 휴게공간이 있다.', '문의 및 안내': '', '홈페이지': 'http://historymuseum.or.kr', '주소': '서울특별시 용산구 청파로47다길 27 (청파동2가)', '이용시간': '10:30~18:00 (입장 마감 17:30)', '휴일': '매주 월요일 / 1월 1일 / 명절 연휴', '주차': '가능', '이용요금': '무료', 'review': '작은 박물관이에요 하지만 초중 아이들과 함께 우리나라의 슬픈 역사를 제대로 알기위해 꼭 방문해보는걸 추천합니다'}
{'place': '서울대학교 미술관', 'info': '서울대학교미술관은 항상 새롭고 독창적이며 시대를 반영하는 예술을 선보이고자 노력해 왔다. 2006년 현대미술로의 초대전을 시작으로 개관한 서울대학교미술관은 동시대 미술의 흐름, 사회적 이슈, 미술관의 역할 등 시대의 변화에 민감하게 반응하여 전시, 행사, 교육프로그램을 제공해 왔다.\n\n국내 최초의 대학미술관인 서울대학교미술관은 그 전신을 1946년에 개관한 서울대학교 박물관 현대미술부에 두고 있다. 박물관 현대미술부는 1995년 서울대학교 50주년 기념사업의 일환으로 미술관 건립 준비를 시작하게 된다. 그해 삼성문화재단이 건립 기증 협찬을 확정하고, 1996년 네덜란드 건축가 렘 쿨하스(Rem Koolhaas)의 설계를

InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff663f9fc95+79861]
	GetHandleVerifier [0x0x7ff663f9fcf0+79952]
	(No symbol) [0x0x7ff663d1c8fc]
	(No symbol) [0x0x7ff663d6438d]
	(No symbol) [0x0x7ff663d9c9a2]
	(No symbol) [0x0x7ff663d97374]
	(No symbol) [0x0x7ff663d9642a]
	(No symbol) [0x0x7ff663ce7075]
	GetHandleVerifier [0x0x7ff66425b9cd+2946349]
	GetHandleVerifier [0x0x7ff664255c4a+2922410]
	GetHandleVerifier [0x0x7ff6642759d7+3052855]
	GetHandleVerifier [0x0x7ff663fbaa7e+189918]
	GetHandleVerifier [0x0x7ff663fc2a1f+222591]
	(No symbol) [0x0x7ff663ce602c]
	GetHandleVerifier [0x0x7ff66438e3f8+4202328]
	BaseThreadInitThunk [0x0x7fff02a8e8d7+23]
	RtlUserThreadStart [0x0x7fff04248d9c+44]
